In [34]:
from itertools import compress
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
import matplotlib.pyplot as plt
%matplotlib notebook

In [35]:
#Path to RDA results
RDA_path_1 = "/Users/Siwen/Desktop/ferret/bin/meiyu_clean.nc"
RDA_path_2 = "/Users/Siwen/Desktop/ferret/bin/meiyu_2_clean.nc"

period = [1,200]
years = [1951,1953]

In [36]:
## dives into the netCDF file and spits out all of the data of particular attribu
def collect_data(years, period):
    
    RDA_1 = nc.Dataset(RDA_path_1, 'r') #all primary events
    RDA_2 = nc.Dataset(RDA_path_2, 'r') #all secondary events

    #load data from NetCDF files to notebook
    lat_1 = RDA_1.variables['lat_115'][:]
    lat_2 = RDA_2.variables['lat_115'][:]

    int_1 = RDA_1.variables['intensity'][:] 
    int_2 = RDA_2.variables['intensity'][:]
    
    #countit_1_all = 0
    #countit_2_all = 0
    
    #Assign a calendar date to each time point
    startday = datetime.datetime(1951,1,1)
    date_list = [datetime.timedelta(days=x) + startday for x in range(0, 20819)]
    
    #the .timetuple().tm_yday command turns a datetime object into a day of the year.
    filter_days = [(dd.timetuple().tm_yday >= period[0]) & (dd.timetuple().tm_yday <= period[1]) & (dd.year >= years[0]) & (dd.year <= years[1]) for dd in date_list]
    days = list(compress(range(20819),filter_days))
    
    #filter full list of latitudes and intensities to only include time period of interest.
    lat_1_out = lat_1[days]
    lat_2_out = lat_2[days]
    int_1_out = int_1[days]
    int_2_out = int_2[days]
    
    lats_out = np.append(lat_1_out, lat_2_out)
    ints_out = np.append(int_1_out, int_2_out)  
    lats_out = lats_out[~np.isnan(lats_out)]
    ints_out = ints_out[~np.isnan(ints_out)] 

    RDA_1.close()
    RDA_2.close()
    
    return [lats_out, ints_out]

In [37]:
collect_data([1961,1970],[181,200])

400
400
0
0


[array([ 38.27510386,  22.50965187,  24.01412259,  34.51625884,
         33.31354689,  31.08145036,  30.96518199,  36.98090893,
         37.27756187,  36.35844503,  23.25113394,  35.28637695,
         25.51342082,  26.8538963 ,  27.22544827,  29.68245108,
         31.3097629 ,  31.39247216,  31.15371652,  32.22021006,
         32.62234772,  32.95693695,  22.60438538,  22.92914581,
         36.38308334,  36.2218895 ,  38.7192344 ,  38.5423729 ,
         35.26363008,  35.12964143,  23.83323738,  34.79184341,
         36.39551446,  33.16873035,  35.50912189,  32.95449878,
         31.33788428,  31.96347683,  30.87684836,  37.93778407,
         36.62642543,  26.31850484,  32.95370776,  33.29954862,
         24.09934281,  37.66591486,  37.24087465,  37.85299483,
         35.76913204,  34.7835667 ,  38.7269372 ,  34.19805613,
         36.57774667,  35.46213234,  33.74460176,  33.17123682,
         32.6196831 ,  32.58381921,  29.91139154,  28.88400164,
         28.89632675,  30.62008354,  32.

In [6]:
startday = datetime.datetime(1951,1,1)
date_list = [datetime.timedelta(days=x) + startday for x in range(0, 20819)]
period = [1,200]
years = [1951,1953]

x = [(dd.timetuple().tm_yday >= period[0]) & (dd.timetuple().tm_yday <= period[1])&\
     (dd.year >= years[0]) & (dd.year <= years[1]) for dd in date_list]

y = list(range(20819))
z = list(compress(y,x))
print(z)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386,